Over the next eight days, New York City Democrats will decide their party nominee for mayor. One of the candidates, former presidential candidate Andrew Yang, has centered his campaign around a shrunk-down local version of his national [Freedom Dividend](https://www.ubicenter.org/distributional-analysis-of-andrew-yangs-freedom-dividend), which he calls ["A Basic Income for New York City."](https://www.yangforny.com/policies/a-basic-income-for-new-york-city) The plan promises to provide an average of $2,000 to each of the city's 500,000 poorest adults.

In this post, I explore how this $1 billion program could be codified. I find that the plan can be characterized by a benefit program that guarantees 30% of the poverty line and phases out at 50 cents per dollar of income. Such a plan would reduce deep poverty by 60%, though it would also create significant work disincentives and distort marriage choices among NYC's deeply poor population.


# Poverty in New York City

NYC has unusually high poverty: in 2019, 19.2% of the population was in poverty, compared to [11.7% nationally](https://www.census.gov/content/dam/Census/library/publications/2020/demo/p60-272.pdf). That's based on the Census Bureau's [Supplemental Poverty Measure](https://www.census.gov/topics/income-poverty/supplemental-poverty-measure.html) (SPM), which, unlike the Official Poverty Measure, accounts for a household's[^1] in-kind benefits (e.g., food stamps), taxes, and, critically for NYC, housing costs—the SPM estimates that NYC residents face 21% higher cost of living than the national average.

[^1]: Throughout this analysis, I use "household" as shorthand for Supplemental Poverty Measure unit, a household-like concept created by the Census Bureau to group (actual) household members that share resources.

NYC's current 19.2% poverty rate is down from 23.9% when the Census Bureau began reporting the SPM, though it hit a high of 28.3% in 2012 in between. The deep poverty rate—the population share under half the poverty threshold—has followed a similar trend, from 7.8% in 2009 to a high of 9.1% in 2011 to its latest value of 4.9% in 2019.

In [1]:
import pandas as pd
import numpy as np
import microdf as mdf
import plotly.express as px
import statsmodels.api as sm
import ubicenter

BUDGET = 1e9  # $1 billion.
REACH = 500e3  # 500,000 adult New Yorkers.
RUN_SIMS = False  # To re-run (takes ~5min).

def format_fig(fig, f):
    fig = ubicenter.format_fig(fig, show=False)
    CONFIG = {"displayModeBar": False}
    fig.write_html(f, config=CONFIG)

# Read data per person per year.
py = pd.read_csv("data/nys_cps.csv.gz")
py.columns = py.columns.str.lower()
py.inctot.replace({999999999: 0}, inplace=True)
py.adjginc.replace({99999999: 0}, inplace=True)
# 2014 was double-sampled.
py.loc[py.year == 2014, ["asecwt", "spmwt"]] /= 2
# ASEC shows survey year.
py.year -= 1

LABELS = {
    "poverty_rate": "Poverty rate",
    "deep_poverty_rate": "Deep poverty rate",
    "adult_deep_poverty_rate": "Deep adult poverty rate",
    "child_deep_poverty_rate": "Deep child poverty rate",
    "poverty_gap": "Poverty gap",
    "squared_poverty_gap": "Poverty severity",
    "deep_poverty_gap": "Deep poverty gap",
    "pov_ratio_guarantee": "Maximum benefit as a share of SPM poverty threshold",
    "phase_out_rate": "Benefit phase-out rate with respect to SPM resources",
    "phase_out_rate_pct": "Benefit phase-out rate with respect to SPM resources",
    "constraint": "Constraint",
    "cost_b": "Cost ($B)",
    "adult_reach": "Adult recipients",
    "pct_diff": "Policy impact",
    "yearly": "Average yearly change, 2009-2019",
    "name": "Metric",
    "years_progress": "Policy impact / average yearly change",
}

LIGHTER_BLUE = "#ABCEEB"  # Blue 100.
LIGHT_BLUE = "#49A6E2"  # Blue 500.
BLUE = "#1976D2"  # Blue 700.
DARK_BLUE = "#0F4AA1"  # Blue 900.
GRAY = "#BDBDBD"

COLOR_MAP = {
    "Poverty rate": LIGHT_BLUE,
    "Deep poverty rate": DARK_BLUE,
    "Poverty gap": LIGHT_BLUE,
    "Deep poverty gap": DARK_BLUE,
    "25%": LIGHTER_BLUE,
    "50%": LIGHT_BLUE,
    "100%": DARK_BLUE,
    "$1 billion cost": GRAY,
    "500,000 adult recipients": BLUE,
}

NYC_COUNTY = pd.DataFrame(columns=["fips", "county", "pop_m"])
NYS_FIPS = 36
# Per https://en.wikipedia.org/wiki/List_of_counties_in_New_York
NYC_COUNTY.loc["Manhattan"] = pd.Series({"fips": 61, "county": "New York", "pop_m": 1.632})
NYC_COUNTY.loc["Brooklyn"] = pd.Series({"fips": 47, "county": "Kings", "pop_m": 2.59})
NYC_COUNTY.loc["The Bronx"] = pd.Series({"fips": 5, "county": "Bronx", "pop_m": 1.435})
NYC_COUNTY.loc["Staten Island"] = pd.Series({"fips": 85, "county": "Richmond", "pop_m": 0.475})
NYC_COUNTY.loc["Queens"] = pd.Series({"fips": 81, "county": "Queens", "pop_m": 2.287})
NYC_COUNTY["full_fips"] = NYS_FIPS * 1000 + NYC_COUNTY.fips

py = py[py.county.isin(NYC_COUNTY.full_fips)][
    ["county", "asecwt", 'age', 'sex', 'race', 'hispan', 'inctot', 'spmwt', 'spmtotres', 'spmthresh',
       'spmfamunit', 'adjginc', 'ftotval', 'spmftotval', 'year', 'spmgeoadj']]
py["adult"] = py.age > 17
py["child"] = py.age <= 17
py["spmratio"] = py.spmtotres / py.spmthresh
py["poor"] = py.spmratio < 1
py["deep_poor"] = py.spmratio < 0.5
NYC_COUNTY["asec_pop_m"] = NYC_COUNTY.full_fips.apply(
    lambda x: py[(py.county == x) & (py.year == py.year.max())].asecwt.sum() / 1e6
)

def year_stats(p):
    s = p[["spmfamunit", "year", "spmtotres", "spmthresh", "spmwt"]].drop_duplicates()
    s["spmtotres_pos"] = np.maximum(s.spmtotres, 0)
    return pd.Series({
        "spmgeoadj": mdf.weighted_mean(p, "spmgeoadj", "asecwt"),
        "poverty_rate": mdf.weighted_mean(p, "poor", "asecwt"),
        "deep_poverty_rate": mdf.weighted_mean(p, "deep_poor", "asecwt"),
        "poor_adults": mdf.weighted_sum(p[p.adult], "poor", "asecwt"),
        "deep_poor_adults": mdf.weighted_sum(p[p.adult], "deep_poor", "asecwt"),
        "child_poverty_rate": mdf.weighted_mean(p[p.child], "poor", "asecwt"),
        "child_deep_poverty_rate": mdf.weighted_mean(p[p.child], "deep_poor", "asecwt"),
        "adult_poverty_rate": mdf.weighted_mean(p[p.adult], "poor", "asecwt"),
        "adult_deep_poverty_rate": mdf.weighted_mean(p[p.adult], "deep_poor", "asecwt"),
        "poverty_gap": mdf.poverty_gap(s, "spmtotres", "spmthresh", "spmwt"),
        "squared_poverty_gap": mdf.squared_poverty_gap(s, "spmtotres", "spmthresh", "spmwt"),
        "deep_poverty_gap": mdf.deep_poverty_gap(s, "spmtotres", "spmthresh", "spmwt"),
        "gini": mdf.gini(s, "spmtotres", "spmwt"),
        "population": p.asecwt.sum(),
        "spm_units": s.spmwt.sum(),
        "survey_population": p.shape[0],
        "survey_spm_units": s.shape[0],
    })
year = py.groupby("year").apply(year_stats)
year["year"] = year.index

pov_rate_trend = year.melt("year", ["poverty_rate", "deep_poverty_rate"])
pov_rate_trend.variable = pov_rate_trend.variable.map(LABELS)
fig = px.line(
    pov_rate_trend,
    "year",
    "value",
    color="variable",
    color_discrete_map=COLOR_MAP,
    labels=LABELS,
    title="Poverty rate trends in NYC (SPM)"
)
fig.update_layout(
    yaxis_tickformat=".1%",
    xaxis_title="",
    yaxis_title="",
    legend_title="",
    yaxis_range=[0, 0.3]
)
format_fig(fig, "pov_trend.html")

Poverty rates fail to capture dynamics below the threshold, though. If a policy lifts someone from 10% of the poverty line to 90%, that doesn't change the poverty rate; if it lifts them from 10% to 40%, it also doesn't change the deep poverty rate.

So it's also useful to consider the _poverty gap_, the sum of all deviations between a household's resources and their poverty threshold, among families in poverty; that is, the poverty gap measures the total amount one would have to spend to end poverty, if the transfer were perfectly targeted. Similarly, the _deep poverty gap _measures deviations to half the poverty threshold.

NYC's poverty gap in 2019 was $6.3 billion, and its deep poverty gap was $1.5 billion. Poverty gaps are less straightforward to interpret over time, since they vary with inflation and population changes, but their pattern roughly mirrors that of the poverty rates, exhibiting modest and inconsistent declines over time.

In [2]:
pov_gap_trend = year.melt("year", ["poverty_gap", "deep_poverty_gap"])
pov_gap_trend.variable = pov_gap_trend.variable.map(LABELS)
fig = px.line(
    pov_gap_trend,
    "year",
    "value",
    color="variable",
    color_discrete_map=COLOR_MAP,
    labels=LABELS,
    title="Poverty gap trends in NYC (SPM)")
fig.update_layout(yaxis_tickprefix="$", xaxis_title="", yaxis_title="", legend_title="",
    yaxis_range=[0, 1e10])
format_fig(fig, "pov_gap_trend.html")


To smooth out the year-by-year noise, the rest of this analysis averages the three latest years of data, from 2017 to 2019.


# Andrew Yang's Basic Income Proposal

Yang's [campaign website](https://www.yangforny.com/policies/a-basic-income-for-new-york-city) spells out core features of the plan, indicating that it would provide monthly transfers averaging $2,000 per year to the "500,000 New Yorkers with the greatest need," that these transfers would supplement existing benefits, and that the $1 billion program intends to end "extreme poverty."

This document leaves some details unspecified, some of which Yang's campaign staff clarified to me, others of which remain open questions:



1. **What is a "New Yorker"?** Campaign staff defined this as an adult resident of NYC, in accordance with [poverty reporting](https://www.robinhood.org/wp-content/themes/robinhood/images/poverty-tracker/pdfs/POVERTY_TRACKER_REPORT25.pdf) from the NYC-based nonprofit [Robin Hood](https://www.robinhood.org/).
2. **How is poverty defined? **Campaign staff also confirmed to me that the poverty definitions refer to the SPM. Given SPM's accounting of housing costs, this aligns with Yang's statement that "Our goal is to [...ensure] that every household has an annual income that is at least above extreme poverty, taking into account the true cost of living in New York City." Robin Hood's Poverty Tracker emulates the Census Bureau's approach with regard to the Supplemental Poverty Measure, though its results differ.
3. **How is "extreme poverty" defined?** This is unclear, so I do not apply it as a defining constraint. Robin Hood's [2019 report](https://www.robinhood.org/wp-content/themes/robinhood/images/poverty-tracker/pdfs/POVERTY_TRACKER_REPORT19.pdf ), based on their [Poverty Tracker](https://www.robinhood.org/programs/special-initiatives/poverty-tracker/index.html) survey produced in conjunction with Columbia University, classified 500,000 NYC adults as in "deep poverty," under the standard poverty definition of below half the poverty line. Meanwhile, the term "extreme poverty" is typically reserved for the World Bank's global definition of about $2 per person per day—a depth of poverty under which [virtually no Americans fall](https://www.nber.org/papers/w25907).
4. **Are the transfers person-based or household-based? **On one hand, the program is focused on "New Yorkers"—adults—but on the other, it promises to end extreme poverty, which is defined at the household level. I assume that it's household-based, like other welfare benefits.
5. **How is income defined for the purposes of qualifying for the benefit? **The emphasis on the poverty rate suggests that the income measure would be the SPM resource measure; indeed, Yang's staff confirmed that benefits like SNAP and housing assistance would count toward eligibility (though the reverse is not true). While estimating this resource measure in real time will create administrative difficulties (as I explain later), I assume that doing so is feasible and use it for my calculations.
6. **What is the maximum amount of the transfer? **That is, how much should a household with no current income get? If the policy intends to eradicate deep poverty, then it must be at least half the household's poverty threshold. As I show in the next section, however, this is not feasible given the $1 billion budget, so I explore smaller amounts; and as I explain later, basing the transfer on a household's poverty threshold may create administrative challenges.
7. **What is the phase-out rate (i.e. marginal tax rate) of the transfer with respect to income? **Alternatively, at what income level would the program no longer cover a household? For example, if the benefit maxes out at half a household's poverty threshold, and it phases out dollar for dollar (100% marginal tax rate), then a household whose income is half their poverty threshold would receive $0; if the marginal tax rate is 50%, they would receive $0 when their income reaches their poverty threshold. The campaign told me that it would phase out at some rate, rather than having a cliff, but they did not specify the rate.

In summary, I assume that the program would guarantee a certain percentage of a household's SPM poverty threshold and phase out at a certain rate with respect to their SPM resources. To determine the parameters that satisfy Yang's constraints ($1 billion budget and 500,000 adult recipients), I turned to microsimulation.


# Identifying a feasible reform

Yang's plan has two criteria: it must reach 500,000 adults and cost $1 billion. There are also two unknown policy parameters: the maximum value (as a share of a household's poverty threshold), and the phase-out rate (with respect to resources). Many potential policies could satisfy either of these conditions, but only one satisfies both.

Consider, for example, three potential phase-out rates: 25%, 50%, and 100%. The slower the phase-out, the more a given policy costs, _ceteris paribus_; UBI, with a 0% phase-out, is the most expensive option, while a 100% phase-out is cheapest (though it removes all incentive to work). At a 25% phase-out, a benefit could only guarantee 20% of the poverty threshold while staying under the $1 billion budget. At a 50% phase-out rate, the benefit could guarantee 30% of the poverty threshold, and a 100% phase-out could guarantee 39%.

In [3]:
# Use three years of data to smooth out given small sample size.
p = py[py.year.isin([2017, 2018, 2019])].copy()
p[["asecwt", "spmwt"]] /= 3

P_S_MERGE_COLS = ["year", "spmfamunit"]  # SPM unit keys from person.
SPMU_COLS = ["spmtotres", "spmthresh", "spmratio", "spmwt", "spmftotval"]
SPMU_AGGS = ["adult", "child"]

s = p.groupby(P_S_MERGE_COLS + SPMU_COLS)[SPMU_AGGS].sum().reset_index()

def phase_out(amount, rate, respect_to):
    return np.maximum(0, amount - respect_to * rate)



def policy(pov_ratio_guarantee, phase_out_rate):
    """ Computes features of a negative income tax.

    Args:
        pov_ratio_guarantee: Maximum benefit as a share of a SPM unit's
            poverty threshold.
        phase_out_rate: Phase-out rate with respect to a SPM unit's resources.
    """
    s["max_transfer"] = s.spmthresh * pov_ratio_guarantee
    s["transfer"] = phase_out(s.max_transfer, phase_out_rate,
                              np.maximum(s.spmtotres, 0))
    s["spmtotres_r"] = s.spmtotres + s.transfer
    s["spmtotres_r_pos"] = np.maximum(s.spmtotres_r, 0)
    p2 = p.merge(
        s[P_S_MERGE_COLS + ["transfer", "spmtotres_r", "spmtotres_r_pos"]],
        on=P_S_MERGE_COLS
    )
    return pd.Series({
        # Variables for defining policies.
        "cost_b": mdf.weighted_sum(s, "transfer", "spmwt") / 1e9,
        "adult_reach": p2[(p2.transfer > 0) & p2.adult].asecwt.sum(),
        # Variables for evaluation.
        "spmu_reach": s[s.transfer > 0].spmwt.sum(),
        "child_reach": p2[(p2.transfer > 0) & p2.child].asecwt.sum(),
        "gini": mdf.gini(s, "spmtotres_r", "spmwt"),
        "poverty_rate": mdf.poverty_rate(p2, "spmtotres_r", "spmthresh", "asecwt"),
        "deep_poverty_rate": mdf.deep_poverty_rate(p2, "spmtotres_r", "spmthresh", "asecwt"),
        "child_poverty_rate": mdf.poverty_rate(p2[p2.child], "spmtotres_r", "spmthresh", "asecwt"),
        "child_deep_poverty_rate": mdf.deep_poverty_rate(p2[p2.child], "spmtotres_r", "spmthresh", "asecwt"),
        "adult_poverty_rate": mdf.poverty_rate(p2[p2.adult], "spmtotres_r", "spmthresh", "asecwt"),
        "adult_deep_poverty_rate": mdf.deep_poverty_rate(p2[p2.adult], "spmtotres_r", "spmthresh", "asecwt"),
        "poverty_gap": mdf.poverty_gap(s, "spmtotres_r_pos", "spmthresh", "spmwt"),
        "squared_poverty_gap": mdf.squared_poverty_gap(s, "spmtotres_r_pos", "spmthresh", "spmwt"),
        "deep_poverty_gap": mdf.deep_poverty_gap(s, "spmtotres_r_pos", "spmthresh", "spmwt"),
    })

if RUN_SIMS:
    policies = mdf.cartesian_product({
        "pov_ratio_guarantee": np.arange(0.01, 0.51, 0.01),
        "phase_out_rate": np.arange(0, 1.01, 0.01)
        })
    policies = pd.concat([policies,
        policies.apply(lambda x: policy(x.pov_ratio_guarantee, x.phase_out_rate), axis=1)],
        axis=1)
    # Takes ~5min to run.
    policies.to_csv("data/policies.csv", index=False)
else:
    policies = pd.read_csv("data/policies.csv")
policies["phase_out_rate_pct"] = (policies.phase_out_rate * 100).astype(int).astype(str) + "%"
policies

fig = px.line(
    policies[policies.phase_out_rate.isin([0.25, 0.5, 1])],
    "pov_ratio_guarantee",
    "cost_b",
    hover_data={"cost_b": ":.2f"},
    color="phase_out_rate_pct",
    color_discrete_map=COLOR_MAP,
    title="Cost of a means-tested benefit in New York City",
    labels=LABELS,
)
fig.add_hline(y=BUDGET / 1e9, line_dash="dash", line_color="gray")
fig.add_annotation(
    x=0.05,
    y=BUDGET / 1e9,
    align="left",
    text="Yang constraint<br>$1 billion cost",
    showarrow=False,
    font_color="gray",
)
fig.update_layout(
    yaxis_tickprefix="$",
    yaxis_ticksuffix="B",
    xaxis_tickformat="%",
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01
    )
)
format_fig(fig, "cost.html")

To reach 500,000 adults, I run the same exercise. At a 25% phase-out, the benefit can only max out at 15% of the poverty threshold. At 50%, it's 30%, and at 100%, it's well over 50%.

In [4]:
fig = px.line(
    policies[policies.phase_out_rate.isin([0.25, 0.5, 1])],
    "pov_ratio_guarantee",
    "adult_reach",
    hover_data={"adult_reach": ":,.0f"},
    color="phase_out_rate_pct",
    color_discrete_map=COLOR_MAP,
    title="Adults receiving a means-tested benefit in New York City",
    labels=LABELS,
)
fig.add_hline(y=REACH, line_dash="dash", line_color="gray")
fig.add_annotation(
    x=0.07,
    y=REACH,
    align="left",
    text="Yang constraint<br>500,000 adult recipients",
    showarrow=False,
    font_color="gray",
)
fig.update_layout(
    yaxis_tickformat=",",
    xaxis_tickformat="%",
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01
    )
)
format_fig(fig, "reach.html")

Combining these constraints, we can map the contours that satisfy each condition. This reveals that the policy that satisfies both (a) guarantees 30% of the poverty line, and (b) phases out at 50 cents per dollar of income.

In [5]:
cost_curve = policies[policies.cost_b < 1].sort_values(
    "cost_b", ascending=False
    ).drop_duplicates("pov_ratio_guarantee").sort_values("pov_ratio_guarantee")
cost_curve["constraint"] = "$1 billion cost"
reach_curve = policies[policies.adult_reach < REACH].sort_values(
    "adult_reach", ascending=False
    ).drop_duplicates("pov_ratio_guarantee").sort_values("pov_ratio_guarantee")
reach_curve["constraint"] = "500,000 adult recipients"
cost_reach_curve = pd.concat([cost_curve, reach_curve])
fig = px.line(
    cost_reach_curve,
    "pov_ratio_guarantee",
    "phase_out_rate",
    hover_data={"cost_b": ":.2f", "adult_reach": ":,.0f"},
    color="constraint",
    color_discrete_map=COLOR_MAP,
    labels=LABELS,
    title="Guaranteed income policies that satisfy Yang's Basic Income for NYC constraints"
)
fig.update_layout(
    xaxis_tickformat="%",
    yaxis_tickformat="%",
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01)
    )
RATIO = 0.3
PS = 0.5
fig.add_annotation(x=RATIO, y=PS,
                   text="Maximum benefit: 30% of poverty threshold<br>Phase-out rate: 50%")
format_fig(fig, "cost_reach.html")

# The reform's impact

Poverty rates are ill-suited to evaluating policies that target the deeply poor population the way this does. Since the reform fully phases out for families with resources of 60% of their poverty threshold, it leaves the poverty rate unchanged. This demonstrates limitations of the poverty rate as a policy evaluation tool: it fails to capture reductions in the extent of poverty.

Even the deep poverty rate largely fails to capture the policy's impact. The benefit is 83% phased out for families at the deep poverty threshold (50% / 60%), so while it does have some effect, the bulk of the impact is in reducing the depth of deep poverty.

The policy most affects the deep poverty gap—lowering it by 60%—which makes sense given it aims to reduce the depth of poverty.

In [10]:
opt = policies[(policies.pov_ratio_guarantee == RATIO) & (policies.phase_out_rate == PS)]
base = policy(0, 0)
comp = pd.DataFrame({"base": base, "reform": opt.iloc[0]}).reset_index().rename(columns={"index": "metric"})

def pct_diff(base, reform):
    return (reform - base) / base

comp["pct_diff"] = pct_diff(comp.base, comp.reform)
# comp["yearly"] = comp.metric.apply(trend)
# comp["years_progress"] = comp.pct_diff / comp.yearly
comp["name"] = comp.metric.map(LABELS)

PLOT_POV_MEASURES = [
    "poverty_rate",
    "deep_poverty_rate",
    "poverty_gap",
    "deep_poverty_gap",
]
fig = px.bar(
    comp[comp.metric.isin(PLOT_POV_MEASURES)].sort_values("pct_diff", ascending=False),
    "pct_diff",
    "name",
    hover_data={"pct_diff": ":%"},
    orientation="h",
    text="pct_diff",
    title="Impact of Basic Income for NYC plan on poverty measures",
    labels=LABELS,
    )
fig.update_traces(marker_color=BLUE, texttemplate='%{text:.0%}')
fig.update_layout(xaxis_tickformat="%", xaxis_title="", yaxis_title="")
# ubicenter.format_fig(fig)
format_fig(fig, "impacts.html")

# Administrative and incentive considerations

While this plan would achieve significant deep poverty reduction, it may also face administrative challenges, and it would certainly create perverse incentives for recipients.

Basing the monthly program on SPM resource values would be difficult. SPM resources include all cash income, as well as in-kind benefits and taxes throughout the year. That means that program officers would have to estimate households' annual income, benefits, and taxes, each month to determine their payment amount. Recipients would face this administrative burden as well, having to report any change to income, benefits, projected tax liability, or household structure to the city. Tax liability aside, this is largely consistent with other benefit programs, though those other benefit programs do fail to reach some of the poor, and these hoops plausibly explain some of those exclusion errors.

The program would add 50 percentage points to recipients' marginal tax rates. According to the [Department of Health and Human Services](https://aspe.hhs.gov/system/files/aspe-files/260661/brief2-overviewmtranalyses.pdf), most households below 75% of the poverty line (as would be targeted by this program) currently face marginal tax rates of 10% or less. However, by changing that from 10% to 60%, or even 0% to 50%, the program would make the deeply poor population some of the highest taxed segment of the population. Studies find that workers in the lowest income decile are the [most sensitive to marginal tax rates](https://www.cbo.gov/sites/default/files/cbofiles/attachments/10-25-2012-Labor_Supply_and_Fiscal_Policy.pdf), suggesting that labor supply would fall as a result.

The program would also distort marriage incentives, both toward and away from it. For example, someone with no income would gain a larger benefit from staying single than from partnering with someone with income; on the other hand, two people with similar income may gain a larger benefit from partnering, depending on their poverty threshold. This also is not a problem that's new either to benefit programs or the [tax code](https://taxfoundation.org/understanding-marriage-penalty-and-marriage-bonus/), but the generosity of the payment and steep phase-out make the issue especially prominent.


# Conclusion

A policy consistent with Andrew Yang's policy outline would reduce deep poverty by 60%, largely helping people escape the deeper depths of poverty that cause lasting harms, especially to children. The sharp means-testing could make it harder to reach eligible recipients, and it will distort those recipients' incentives, but providing cash transfers without a work requirement across NYC would constitute a new step toward an unconditional safety net.